
#### 9.2 원하는 숫자 생성

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("..\data\mnist\data")

Extracting ..\data\mnist\data\train-images-idx3-ubyte.gz
Extracting ..\data\mnist\data\train-labels-idx1-ubyte.gz
Extracting ..\data\mnist\data\t10k-images-idx3-ubyte.gz
Extracting ..\data\mnist\data\t10k-labels-idx1-ubyte.gz


In [3]:
###########
# 옵션 설정
###########
total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10


##############
# 신경망 모델 구성
#############
X = tf.placeholder(tf.float32, [None, n_input])
# 노이즈에 레이블 데이터를 힌트로 넣어줌. 
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])


# 생성자 신경망 구성
# 학습 시 각 신경망의 변수들을 따로 학습시킨다.
def generator(noise, labels):
    # tf.layers 사용 -> scope 지정 : scope에 해당하는 변수들만 불러올 수 있다.
    with tf.variable_scope('generator'):
        #concat 함수 : noise값에 labels 정보를 추가
        inputs = tf.concat([noise, labels], 1)
        # 은닉층 생성
        hidden = tf.layers.dense(inputs, n_hidden,
                                activation = tf.nn.relu)
        # 출력층 구성
        output = tf.layers.dense(hidden, n_input,
                                activation = tf.nn.sigmoid)
    return output


        
# 구분자 신경망 구성
def discriminator(inputs, labels, reuse=None):
    # 진짜 이미지를 판별할 때와 가짜 이미지를 판별할때 똑같은 변수 사용해야함.
    # scope.reuse_variables 함수로 이전에 사용한 변수 재사용
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden,
                                activation = tf.nn.relu)
        # 손실값에 sigmoid_entropy 함수 사용하기 위해 활성화함수 안씀
        output = tf.layers.dense(hidden, 1,
                                activation = None)
    return output



# 무작위 노이즈 생성 함수
def get_noise(batch_size, n_noise):
    return np.random.normal(-1., 1., size=(batch_size, n_noise))



# 레이블 정보 추가 -> 레이블에 해당하는 이미지 생성 유도
G = generator(Z, Y)
D_real = discriminator(X, Y)
# 가짜 이미지 구분자 -> 진짜 이미지 구분에 사용한 변수 사용하도록 reuse=True
D_gene = discriminator(G, Y, True)



In [4]:


###########
# 손실함수
###########
# D_real : 1에 가깝게, 실제 이미지는 진짜라고 판별
# D_gene : 0에 가깝게, 가짜 이미지는 가짜라고 판별
# sigmoid_cross_entropy_with_logits 함수 사용

loss_D_real = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(
            # ones_like : D_real 결과값과 D_real 크기만큼 1로 채운 값들을 비교
            logits=D_real, labels=tf.ones_like(D_real)))

loss_D_gene = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(
            logits=D_gene, labels=tf.ones_like(D_gene)))

# 이 값을 최소화하면 구분자(경찰)을 학습시킬 수 있다.
loss_D = loss_D_real + loss_D_gene



# loss_G ㅣ 생성자 학습시키기 위한 손실값
loss_G = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(
            logits=D_gene, labels=tf.ones_like(D_gene)))



In [5]:

# discriminator, generator 스코프에서 사용된 변수들을 가져온 뒤
# 이 변수들을 최적화할 각 손실함수와 함께 최적화 함수에 넣어
# 학습 모델 구성 완료
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                          scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                          scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D,
                                           var_list = vars_D)

train_G = tf.train.AdamOptimizer().minimize(loss_G,
                                           var_list = vars_G)




###########
# 학습 진행
###########
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0


for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D],
                                feed_dict={X: batch_xs, Y:batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                feed_dict={Y: batch_ys, Z: noise})
    print('Epoch::','%04d' % epoch,
         'D loss:: {:.4}'.format(loss_val_D),
         'G loss:: {:.4}'.format(loss_val_G))
    

    #############
    # 결과 확인 (for문 안에서)
    # 0, 9, 19, 29..마다 생성기로 이미지를 생성하여 확인
    ############

    # 노이즈를 만들고 생성자 G에 넣어 결과값을 만든다.
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Y:mnist.test.labels[:sample_size],
                                         Z: noise})


    # 노이즈 결과값을 28*28 크기의 가짜 이미지로 만들고
    fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))
    
    # 위 : 진짜 이미지, 아래 : 생성한 이미지
    for i in range(sample_size):
        ax[0][i].set_axis_off()
        ax[1][i].set_axis_off()
        
        ax[0][i].imshow(np.reshape(mnist.test.images[i], (28,28)))
        ax[1][i].imshow(np.reshape(samples[i], (28,28)))
    # samples 폴더에 저장
    plt.savefig('samples2\{}.png'.format(str(epoch).zfill(3),
                                       bbox_inches='tight'))

    plt.close(fig)
    
print('최적화완료!')

ValueError: Cannot feed value of shape (100,) for Tensor 'Placeholder_1:0', which has shape '(?, 10)'